# Implementing Basic RAG QA with Langchain
This notebook demonstrates how to implement a basic Retrieval-Augmented Generation (RAG) chain using our FAQ data. The overall approach is as follows:
1. Load FAQ Data & Structure Metadata
2. Create Embeddings for the FAQ Data using Amazon's Titan Embeddings model, and save these embeddings locally using Chroma.
3. Create a Question Answering (QA) chain which retrieves context based on the embeddings saved in Chroma, serving these as context to the Amazon Titan Express LLM to answer the provided user prompt.
4. Format the response so that source materials can be cited.

This implementation mostly follows these Langchain tutorials:
- https://python.langchain.com/docs/modules/data_connection/document_loaders/json#using-jsonloader

In [6]:
# Define metadata extraction function so we can filter on sections and return deep links as sources
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["section"] = record.get("section")
    metadata["source"] = record.get("deep_link")
    
    return metadata

In [7]:
# Import JSON FAQ File using JSONLoader
from langchain_community.document_loaders import JSONLoader
from pprint import pprint

file_path='../data/processed/faq_data/EN_SYR.json'

loader = JSONLoader(
    file_path=file_path,
    jq_schema=".[]",
    content_key="answer",
    metadata_func=metadata_func
)

data = loader.load()

In [8]:
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.vectorstores import Chroma

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/EN_SYR'
vectorstore = Chroma.from_documents(documents=data, embedding=embeddings, persist_directory=persistDirectory)
vectorstore.persist()

The following cell uses already-persisted ChromaDB Embeddings.

In [74]:
from langchain_community.embeddings import BedrockEmbeddings

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v1", region_name="us-east-1"
)

persistDirectory = './data/processed/faq_data/vectordata/EN_SYR'
# Retrieve and generate answers using relevant FAQs
vectordb = Chroma(persist_directory=persistDirectory,
                  embedding_function=embeddings)



In [75]:
from utils import CreateInferenceModifier # Import the function from utils.py
from langchain import hub
from langchain_community.llms import Bedrock
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

retriever = vectordb.as_retriever()

prompt_template = """Drawing from your knowledge base, Answer the user's question.
If you don't know the answer from the provided context, say that your training materials don't include this information.
Keep the answer as concise and relevant to the user's question as possible.

Knowledge Base: {context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(prompt_template)


# Define the universal set of modifier parameters
modifiers = {"max_tokens": 4000,
             "temperature": 0.2,
             "top_k": 250,
             "top_p": 1,
             "stop_sequences": ["\n\nHuman"],
             }


llm = Bedrock(model_id="anthropic.claude-v2:1",
              model_kwargs=CreateInferenceModifier("claude", params=modifiers))

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [76]:
# Define a function to extract unique URLs used in the retrieved source materials.
def extract_unique_urls(response):
    unique_urls = set()  # Use a set to store unique URLs
    
    # Iterate through each document in the 'context'
    for document in response['context']:
        source_url = document.metadata['source']  # Extract the 'source' URL
        unique_urls.add(source_url)  # Add the URL to the set
    
    # Convert the set of unique URLs to a string
    urls_string = '; '.join(unique_urls)
    
    return urls_string

# Invoke the chain and print the response and sources.
response = rag_chain_with_source.invoke("")
answer = response["answer"]
prettyResponse = f"## Answer: \n\n {answer} \n\n ## Sources: \n\n {extract_unique_urls(response)}"


ValueError: Error raised by inference endpoint: An error occurred (ValidationException) when calling the InvokeModel operation: Malformed input request: expected minLength: 1, actual: 0, please reformat your input and try again.

In [72]:
from IPython.display import Markdown, display
Markdown(prettyResponse)

## Answer: 

  Yes, it is possible to re-apply for temporary protection in Turkey after voluntarily returning to Syria and then coming back to Turkey. However, your case will go through an individualized assessment process. 

You would need to apply at the Provincial Directorate of Migration Management where you were originally registered. You may be asked to submit a petition explaining why you returned and are seeking asylum again. You can also provide any supporting documents you have. Authorities may interview you as well to understand your specific circumstances.

Based on this process, a positive or negative decision will be made on your temporary protection application. During this, you can seek help from NGOs or request legal aid services from the bar association. 

 ## Sources: 

 https://multecihaklari.info/services/registration-status/?section=questions&question=34